In [1]:
import pandas as pd
import tensorflow as tf

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv(Path('..', 'data', 'alzheimers_disease_data.csv'))

In [3]:
# Drop unnecessary columns
df.drop(columns=['PatientID', 'DoctorInCharge'], inplace=True)

In [4]:
# Split preprocessed data into features and target arrays
X = df.drop(columns='Diagnosis')
y = df['Diagnosis']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Scale the data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Determine the number of input neurons
print(f'Input neurons: {len(X_train_scaled[0])}')

Input neurons: 32


In [5]:
# Define the model
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Input(shape=(32,)))
nn.add(tf.keras.layers.Dense(units=2, activation='tanh'))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 2)              │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69 (276.00 B)

 Trainable params: 69 (276.00 B)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

results = []
epochs_trained = 0

for i in range(30):
    # Train the model
    trained_model = nn.fit(
        X_train_scaled, 
        y_train, 
        epochs=5
    )

    epochs_trained += len(trained_model.epoch)

    # Evaluate the model using the test data
    model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
    results.append({
        'Epochs Trained' : epochs_trained,
        'Accuracy' : model_accuracy,
        'Loss' : model_loss
    })

results_df = pd.DataFrame(results)
results_df.set_index('Epochs Trained', inplace=True)
results_df['Acurracy - Loss'] = results_df['Accuracy'] - results_df['Loss']
results_df.sort_values('Acurracy - Loss', ascending=False, inplace=True)

Epoch 1/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4995 - loss: 0.7644 
Epoch 2/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5379 - loss: 0.7307
Epoch 3/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5588 - loss: 0.6922
Epoch 4/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6032 - loss: 0.6673
Epoch 5/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6334 - loss: 0.6438
17/17 - 0s - 9ms/step - accuracy: 0.6580 - loss: 0.6365
Epoch 1/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6756 - loss: 0.6086
Epoch 2/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6914 - loss: 0.5968
Epoch 3/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7189 - loss: 0.5740
Epoch 4/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7251 - loss: 0.5597
Epoch 5/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7171 - loss: 0.5515
17/17 - 0s - 2ms/step - accuracy: 0.7472 - loss: 0.5322
Epoch 1/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accura

In [7]:
print('Maximum accuracy: {} at {} epochs'.format(round(results_df['Accuracy'].max(), 4), results_df['Accuracy'].idxmax()))
print('Minimum loss:     {} at {} epochs'.format(round(results_df['Loss'].min(), 4), results_df['Loss'].idxmin()))
results_df

Maximum accuracy: 0.8587 at 20 epochs
Minimum loss:     0.3737 at 35 epochs


,Accuracy,Loss,Acurracy - Loss
Epochs Trained,,,
35,0.853160,0.373744,0.479416
30,0.855019,0.375816,0.479203
40,0.853160,0.374203,0.478956
50,0.853160,0.375015,0.478145
55,0.853160,0.375344,0.477816
60,0.853160,0.376584,0.476576
65,0.853160,0.376604,0.476556
45,0.851301,0.374989,0.476312
70,0.853160,0.377504,0.475655
